In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
!pip install py-sudoku
!pip uninstall transformers -y
!pip install transformers -U
!pip install umap-learn

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install nnsight
!pip install captum --no-deps

In [1]:
import seaborn as sns
from sklearn.linear_model import LogisticRegression as LOG, LinearRegression as LR, Ridge
from sklearn.model_selection import cross_validate as cv, cross_val_predict as cvp
from sklearn.multioutput import MultiOutputClassifier as MOC
from sklearn.tree import DecisionTreeClassifier as DTC
import sklearn
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sudoku import Sudoku
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
from sklearn.metrics import *
import seaborn as sns
from nnsight import NNsight
from umap import UMAP
import time
import random
import warnings
import gc
from nnsight import LanguageModel, util
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe" 
from nnsight import LanguageModel, util
import einops
# Suppress specific FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

try:
    del model
    gc.collect()
except:
    pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

model_id = "answerdotai/ModernBERT-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

model = model.to(device)

def get_model():
    try:
        del model
        gc.collect()
        torch.cuda.empty_cache()
    except:
        pass
        
    gc.collect()
    torch.cuda.empty_cache()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

    model_id = "answerdotai/ModernBERT-large"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForMaskedLM.from_pretrained(model_id)
    
    model = model.to(device)
    return model

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Generate Sudoku puzzles

In [ ]:
import numpy as np
from tqdm import tqdm
from sudoku import Sudoku

# Create a 9x9 mask matrix
zero = np.zeros((9, 9), dtype=int)


zero[4, 4] = 1
def get_board():
    puzzle = Sudoku(3, seed=np.random.rand()).difficulty(10**(-9))
    a = np.array(puzzle.board)
    #a*=zero

    a = a.astype(str)

    a[a == "0"] = "[PAD]"#random_numbers[array == 0]  # Replace zeros
    
    center = a[4,4].copy()
    
    a[4, 4] = '[MASK]'
    #return a, int(center)
    
    prompt = '\n'.join(' '.join(row) for row in a)
    
    return prompt, int(center)

X, y = [], []

for _ in tqdm(range(512)):
    prompt, center = get_board()
    X.append(prompt)
    y.append(center)


Normal prediction of sudoku puzzles

In [ ]:
def get_prediction(prompt):
  with torch.no_grad():
    text = prompt
    inputs = tokenizer(text, return_tensors="pt")
    for i in inputs:
      inputs[i] = inputs[i].to(device)

    outputs = model(**inputs)

    masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
    print(masked_index)
    predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
    predicted_token = tokenizer.decode(predicted_token_id)

    if predicted_token in ['\n', '[SEP]', ',', '[UNK]']:
        predicted_token = "0"
    return int(predicted_token)

pred = []
for p in tqdm(X[:128]):
  pred.append(get_prediction(p))

In [ ]:
lim=128
print(classification_report(y[:lim], pred))
print(balanced_accuracy_score(y[:lim], pred))

In [ ]:
pred[0]

Generate perturabtion

In [ ]:
def generate_perturb(a):
    #a[4, 4] = '[MASK]'
        
    prompt = '\n'.join(' '.join(row) for row in a)
    
    puzz = [prompt]
    
    for i in range(9):
        for j in range(9):
            board = a.copy()
            board[i][j]="[PAD]"
            board[4,4]='[MASK]'
    
            prompt = '\n'.join(' '.join(row) for row in board)
            puzz.append(prompt)
    return puzz


puzz = generate_perturb(X[0])

In [ ]:
arr=[]
for i in tqdm(range(30)):
    puzz = generate_perturb(X[i])
    local = []
    for p in puzz:
        with torch.no_grad():
            text = p
            inputs = tokenizer(text, return_tensors="pt")
            for j in inputs:
              inputs[j] = inputs[j].to(device)
        
            outputs = model(**inputs)
        
            masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
            #print("Location of mask:", masked_index)
            predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
            predicted_token = tokenizer.decode(predicted_token_id)

            if predicted_token in ["\n", ',', '[SEP]']:
                predicted_token = -1
                
            local.append(predicted_token)
    arr.append(local)

In [ ]:
arr = pd.DataFrame(arr).astype(int)#.replace(-1, -1)

diff = arr-arr[0].values.reshape((-1,1))
diff[abs(diff)>0]=1

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.heatmap(abs(diff).sum().values[1:].reshape(9,9), cmap='coolwarm', center=0, linewidth=1)

for i in range(9):  # Rows
    for j in range(9):  # Columns
        ax.text(j + 0.5, i + 0.5, X[0][i, j], 
                ha='center', va='center', color='black', fontsize=12)
        
plt.title("Cells when replaced with [PAD] change model prediction ")
plt.xlabel("Columns")
plt.ylabel("Row")

Using the MASK activation for visulisation and linear probe

In [ ]:

res = []
layer=[]
res_y = []

for j in tqdm(range(256)):
    p = X[j]
    
    with torch.no_grad():
        text = p
        inputs = tokenizer(text, return_tensors="pt")
        for i in inputs:
          inputs[i] = inputs[i].to(device)
    
        outputs = model(**inputs, output_hidden_states=True).hidden_states

    for c,i in enumerate(outputs):
        res.append(i[0, 45].detach().cpu())
        layer.append(c)
        res_y.append(y[j])

res = np.array(res)
layer = np.array(layer)
res_y = np.array(res_y)



In [ ]:
acc = []
for lr in range(29):
    loc = res[layer==lr]
    loc_y = res_y[layer==lr]

    loc_cv = cv(LOG(max_iter=10000), loc, loc_y, scoring='f1_weighted', cv=3)
    print(lr, loc_cv['test_score'].mean())
    acc.append(loc_cv['test_score'].mean())

In [ ]:
plt.figure(figsize=(10, 8))
sns.lineplot(acc)
plt.title("F1 score of logistic regression using the [MASK] activation per layer.")
plt.xlabel("Layer")
plt.ylabel("F1 weighted")

In [ ]:
res.shape

In [ ]:
ump = UMAP(verbose=2).fit(res)

In [ ]:
image_files=[]
for lr in range(29):
    ump = UMAP(verbose=2).fit_transform(res[layer==lr])

    scatter = plt.scatter(x=ump[:, 0], y=ump[:, 1], c=res_y[layer==lr], cmap='inferno')

    plt.colorbar(scatter, label='[MASK] true value')
    
    plt.title(f"Layer {lr} UMAP of the [MASK] activation")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    filename = f"plot_{lr}.png"
    plt.savefig(filename)
    image_files.append(filename)

    plt.clf()

In [ ]:
import imageio
import os
# Create a GIF from the images
with imageio.get_writer('plot_animation.gif', mode='I', duration=100, loop=0) as writer:
    for filename in image_files:
        for i in range(10):
            image = imageio.imread(filename)
            writer.append_data(image)



In [ ]:
for filename in image_files:
    os.remove(filename)

In [ ]:
sns.scatterplot(x=emb[:, 0], y=emb[:, 1], hue=res_y, palette='coolwarm')

Attention map visualisation of the tokens across the model layers

In [ ]:
atts=[]

for j in tqdm(range(100)):
    p = X[j]
    prompt = '\n'.join(' '.join(row) for row in p)
    
    with torch.no_grad():
        text = prompt
        inputs = tokenizer(text, return_tensors="pt")
        for i in inputs:
          inputs[i] = inputs[i].to(device)
    
        outputs = model(**inputs, output_attentions=True).attentions
        outputs = torch.stack(outputs)

    outputs = outputs[:,0,:,45].mean(dim=1).detach().cpu().numpy()
    atts.append(outputs)

atts = np.array(atts)#.mean(axis=0)

In [ ]:
atts = (atts - atts.min())/(atts.max()-atts.min())

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(outputs, center=0,  linewidths=0)#, cmap='coolwarm')

Easter Egg: Average attention score over the layers, grouped by function (row, column, box, other)
My investigation showed row as having the most attention, followed box(!!), and surprisingly other and column have similar attention scores.

In [ ]:
import numpy as np

row = atts[:, :, [41, 42, 43, 44, 46, 47, 48, 49]]  # Slices indices 41 to 50 along the third axis (column)
col = atts[:, :, [5, 15, 25, 35, 55, 65, 75, 85]]  # Specific column indices
box = atts[:, :, [34, 35, 36, 44, 46, 54, 55, 56]]  # Specific box indices

exclude = [5, 15, 25, 35, 45, 55, 65, 75, 85, 34, 35, 36, 44, 45, 46, 54, 55, 56, 41,42,43,44,45,46,47,48,49, 0,10,20,30,40,50,60,70,80,90]

lin = np.linspace(0, 90, 91)
new=[]
for i in lin:
    if i in exclude:
        pass
    else:
        new.append(int(i))
        
oth = atts[:, :, new]

In [ ]:
for i in range(10):
    r_m = row[i].mean(axis=1)
    c_m = col[i].mean(axis=1)
    b_m = box[i].mean(axis=1)
    oth_m = oth[i].mean(axis=1)

In [ ]:
r_m   = row.mean(axis=(0,2))
c_m   = col.mean(axis=(0,2))
b_m   = box.mean(axis=(0,2))
oth_m = oth.mean(axis=(0,2))

In [ ]:
m = pd.DataFrame([r_m, c_m, b_m, oth_m])

In [ ]:
sns.boxplot(m.T)

In [ ]:
sns.lineplot(data=r_m, label="r_m")
sns.lineplot(data=c_m, label="c_m")
sns.lineplot(data=b_m, label="b_m")
sns.lineplot(data=oth_m, label="oth_m")

In [ ]:
time.sleep(10000)

Violation identification with linear probe

In [ ]:
import numpy as np
from tqdm import tqdm
from sudoku import Sudoku

# Create a 9x9 mask matrix
zero = np.ones((9, 9), dtype=int)
#zero[1:8, :] =0
#zero[:, 2]=1
#zero[4:5, :] = 0
#zero[:, 4:5] = 1
#zero[:, :1]=1
#zero[3:6, 3:6] = 0
#zero[4:5]=1

zero[4, 4] = 1
def get_board():
    puzzle = Sudoku(3, seed=np.random.rand()).difficulty(10**(-9))
    a = np.array(puzzle.board)
    #a*=zero
    a = a.astype(str)

    #random_numbers = np.random.randint(1, 10, size=a.shape)  # Random numbers from 1 to 9
    #a[a == "0"] = "[PAD]"#random_numbers[array == 0]  # Replace zeros

    center = a[4,4]
    a[4, 4] = '[MASK]'
    
    #return a, int(center)
    prompt = '\n'.join(' '.join(row) for row in a)
    
    return prompt, int(center)

def get_violation():
    puzzle = Sudoku(3, seed=np.random.rand()).difficulty(10**(-9))
    a = np.array(puzzle.board)
    #a*=zero
    a = a.astype(str)

    #random_numbers = np.random.randint(1, 10, size=a.shape)  # Random numbers from 1 to 9
    #a[a == "0"] = "[PAD]"#random_numbers[array == 0]  # Replace zeros
    a[7,0]=a[7,1]
    center = a[4,4]
    a[4, 4] = '[MASK]'
    
    #return a, int(center)
    prompt = '\n'.join(' '.join(row) for row in a)
    
    return prompt, int(center)

X, y = [], []

for _ in tqdm(range(32)):
    prompt, center = get_board()
    X.append(prompt)
    y.append(center)

for _ in tqdm(range(32)):
    prompt, center = get_violation()
    X.append(prompt)
    y.append(center)

valid = [0]*32 + [1]*32

In [ ]:
print(X[0])

In [ ]:
print(X[32])

In [ ]:
X_v=[]
y_v=[]
layer=[]

for j in tqdm(range(64)):
    p = X[j]
    
    #prompt = '\n'.join(' '.join(row) for row in p)
    
    with torch.no_grad():
        text = p
        inputs = tokenizer(text, return_tensors="pt")
        for i in inputs:
          inputs[i] = inputs[i].to(device)
    
        outputs = model(**inputs, output_hidden_states=True).hidden_states
        outputs = torch.stack(outputs)

    outputs = outputs[:, 0, 75].detach().cpu().numpy()
    for c,i in enumerate(outputs):
        X_v.append(i)
        layer.append(c)
        y_v.append(valid[j])


X_v = np.array(X_v).reshape((-1, 1024))
y_v = np.array(y_v).flatten()
layer = np.array(layer).flatten()


In [ ]:
print(X_v.shape, y_v.shape, layer.shape)

In [ ]:
acc = []
for lr in range(28):
    loc = X_v[layer==lr]
    loc_y = y_v[layer==lr]

    loc_cv = cv(LOG(max_iter=1000), loc, loc_y, scoring='balanced_accuracy', cv=3)
    #lg = XG(device='cuda').fit(loc, loc_y)
    
    print(lr, loc_cv['test_score'].mean())
    acc.append(loc_cv['test_score'].mean())

In [ ]:
dic={}

for token in range(60,80):
    print(token)
    X_v=[]
    y_v=[]
    layer=[]
    
    for j in tqdm(range(64)):
        p = X[j]
        
        #prompt = '\n'.join(' '.join(row) for row in p)
        
        with torch.no_grad():
            text = p
            inputs = tokenizer(text, return_tensors="pt")
            for i in inputs:
              inputs[i] = inputs[i].to(device)
        
            outputs = model(**inputs, output_hidden_states=True).hidden_states
            outputs = torch.stack(outputs)
    
        outputs = outputs[:, 0, token].detach().cpu().numpy()
        for c,i in enumerate(outputs):
            X_v.append(i)
            layer.append(c)
            y_v.append(valid[j])
    
    
    X_v = np.array(X_v).reshape((-1, 1024))
    y_v = np.array(y_v).flatten()
    layer = np.array(layer).flatten()

    acc = []
    for lr in range(28):
        loc = X_v[layer==lr]
        loc_y = y_v[layer==lr]
    
        loc_cv = cv(LOG(max_iter=1000), loc, loc_y, scoring='f1', cv=3)
        #lg = XG(device='cuda').fit(loc, loc_y)
        
        print(lr, loc_cv['test_score'].mean())
        acc.append(loc_cv['test_score'].mean())

    dic[token] = acc

In [ ]:
tot = pd.DataFrame(dic)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20, 18))
counter=0

for i in range(2):
    for j in range(2):
        bip = tot.iloc[counter].values[1:91].reshape(9,10)[:, :9]
        bip = np.where(bip < 0.6, 0.5, bip)
        sns.heatmap(bip, ax=ax[i][j], center=0.5, vmin=0, vmax=1, cmap='coolwarm')
        ax[i][j].set_title(f"Layer_{counter}", fontsize=24)
        counter+=7
